In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
# setup
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import pandas as pd
import io

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving comments_annotated_1.xlsx to comments_annotated_1.xlsx


In [ ]:
data = pd.read_excel('comments_annotated_1.xlsx')

In [ ]:
data.head()

,comment_user,comment_time,reply_count,post_comment_count,Channel,comment,title,comment_id,language,label
0,@theblissfulgiantsfan5627,NaN,7,"1,106",Trump,tulsi gabbard is a strong woman supporting trump,MAGA Women Respond to Kamala Surrogate Mark Cuban,91179,en,positive
1,@roxannakarn6072,NaN,0,954,Trump,beautiful sight \ni love a president thats a r...,"President Trump's Grandchildren, Luke and Caro...",9196,en,positive
2,@BrandonDutchak-dk7lq,NaN,0,"2,399",Harris,make higher education more affordable or free ...,Say It To My Face | Harris-Walz 2024,53661,en,neutral
3,@gedigi9010,2024-11-07 06:54:00,0,"1,543",Harris,oprah that was beautiful,Oprah Winfrey Welcomes First-Time Voters To A ...,61118,en,positive
4,@cecewhite952,NaN,0,"3,464",Trump,why dont they mute the video so we can hear pr...,President Trump Reacts To Kamala's DNC Speech ...,102257,en,neutral


In [ ]:
data = data[['comment', 'label']]
data = data.rename(columns={'comment': 'text'})

In [ ]:
type(data['text'][0])

str

In [ ]:
data.head()

,text,label
0,tulsi gabbard is a strong woman supporting trump,positive
1,beautiful sight \ni love a president thats a r...,positive
2,make higher education more affordable or free ...,neutral
3,oprah that was beautiful,positive
4,why dont they mute the video so we can hear pr...,neutral


In [ ]:
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
data['label'] = data['label'].map(label_mapping)

# Check for missing or invalid mappings
if data['label'].isnull().any():
    print("Found missing or invalid labels. Rows with invalid labels will be removed.")
    # Drop rows with invalid labels
    data = data.dropna(subset=['label'])

# Convert to integers
data['label'] = data['label'].astype(int)

# Verify data types
print(data['label'].dtype)

Found missing or invalid labels. Rows with invalid labels will be removed.
int64


<ipython-input-33-3ea9412cff49>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data['label'].astype(int)


In [ ]:
data.head()

,text,label
0,tulsi gabbard is a strong woman supporting trump,0
1,beautiful sight \ni love a president thats a r...,0
2,make higher education more affordable or free ...,2
3,oprah that was beautiful,0
4,why dont they mute the video so we can hear pr...,2


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['text'], data['label'], test_size=0.1, random_state=42
)

In [ ]:
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})
dataset = DatasetDict({'train': train_dataset, 'validation': val_dataset})

In [ ]:
print(type(train_texts[0]), train_texts[0])

<class 'str'> tulsi gabbard is a strong woman supporting trump


load tokenizer and pre-trained model

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['text'])  # Keep only tokenized inputs and labels


Map:   0%|          | 0/985 [00:00<?, ? examples/s]

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 classes
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Define Training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Adjust based on available time
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",  # Disable reporting to W&B or similar services
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install evaluate

In [ ]:
from evaluate import load
accuracy = load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-47-8dc601782a9a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Fine tuning of the model

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.994700,0.699493,0.700000
2,0.790700,0.630667,0.690909
3,0.632500,0.615931,0.681818


TrainOutput(global_step=186, training_loss=0.7609953008672242, metrics={'train_runtime': 380.2342, 'train_samples_per_second': 7.772, 'train_steps_per_second': 0.489, 'total_flos': 777500149386240.0, 'train_loss': 0.7609953008672242, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

print("Fine-tuning complete. Model saved!")

Fine-tuning complete. Model saved!


In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
encoded_val = tokenizer(val_texts.tolist(), padding=True, truncation=True, return_tensors="pt")
encoded_val = {key: val.to(device) for key, val in encoded_val.items()}

In [ ]:
val_labels = val_labels.tolist()


In [ ]:
with torch.no_grad():
    outputs = model(**encoded_val)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
accuracy = accuracy_score(val_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(val_labels, predictions, average='weighted')

# Print results
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1 Score: {f1:.4f}")

Validation Accuracy: 0.7000
Validation Precision: 0.5838
Validation Recall: 0.7000
Validation F1 Score: 0.6293


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


apply classifier to all comments


In [ ]:
uploaded = files.upload()


Saving comments_only.csv to comments_only.csv


In [ ]:
data = pd.read_csv('comments_only.csv')

In [ ]:
data.head()

,comment_user,comment_time,reply_count,post_comment_count,Channel,comment,title,comment_id,language
0,@JConlon96,2024-11-07 09:54:00,0.0,486,Harris,harris,Big Things | Harris-Walz 2024,25828.0,en
1,@BookMama72,2024-11-07 14:54:00,11.0,480,Harris,im nauseously optimistic this morning lets do ...,Girl Dad | Harris-Walz 2024,41128.0,en
2,@ColeFNCS,2024-11-07 21:54:00,1.0,480,Harris,just voted ballot red lets go america trump i...,Girl Dad | Harris-Walz 2024,46144.0,en
3,@bryanpotter3882,2024-11-07 20:54:00,0.0,413,Harris,kamala harris for president,Good Sense | Harris-Walz 2024,46926.0,no
4,@rhogenelabata2774,2024-11-07 16:54:00,0.0,413,Harris,kentuckian republican here for harriswalz,Good Sense | Harris-Walz 2024,48831.0,en


In [ ]:
data = data.dropna(subset=['comment'])

# Convert comments to strings (if necessary)
comments = data['comment'].astype(str).tolist()

# Tokenize the comments
encoded_comments = tokenizer(comments, padding=True, truncation=True, return_tensors="pt")
encoded_comments = {key: val.to(device) for key, val in encoded_comments.items()}

In [ ]:
# Define a function to process comments in smaller batches
def predict_in_batches(comments, model, tokenizer, batch_size=32, device="cuda"):
    predictions = []
    model.eval()  # Ensure model is in evaluation mode

    # Process in batches
    for i in range(0, len(comments), batch_size):
        batch_comments = comments[i:i + batch_size]

        # Tokenize the batch
        encoded_batch = tokenizer(batch_comments, padding=True, truncation=True, return_tensors="pt")
        encoded_batch = {key: val.to(device) for key, val in encoded_batch.items()}

        with torch.no_grad():
            outputs = model(**encoded_batch)
            logits = outputs.logits
            batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()

        predictions.extend(batch_predictions)

    return predictions

# Apply the function to predict on all comments
comments = data['comment'].astype(str).tolist()
predicted_labels = predict_in_batches(comments, model, tokenizer, batch_size=32, device=device)

# Map predictions back to their labels
label_mapping = {0: 'positive', 1: 'negative', 2: 'neutral'}
predicted_labels = [label_mapping[pred] for pred in predicted_labels]

# Add predictions to the DataFrame
data['predicted_label'] = predicted_labels

# Save the updated DataFrame to a CSV file
data.to_csv('classified_comments.csv', index=False)
print("Classified comments saved to 'classified_comments.csv'")


Classified comments saved to 'classified_comments.csv'


In [ ]:
calssified_comments = pd.read_csv('classified_comments.csv')

In [ ]:
calssified_comments.head()

,comment_user,comment_time,reply_count,post_comment_count,Channel,comment,title,comment_id,language,predicted_label
0,@JConlon96,2024-11-07 09:54:00,0.0,486,Harris,harris,Big Things | Harris-Walz 2024,25828.0,en,positive
1,@BookMama72,2024-11-07 14:54:00,11.0,480,Harris,im nauseously optimistic this morning lets do ...,Girl Dad | Harris-Walz 2024,41128.0,en,positive
2,@ColeFNCS,2024-11-07 21:54:00,1.0,480,Harris,just voted ballot red lets go america trump i...,Girl Dad | Harris-Walz 2024,46144.0,en,positive
3,@bryanpotter3882,2024-11-07 20:54:00,0.0,413,Harris,kamala harris for president,Good Sense | Harris-Walz 2024,46926.0,no,positive
4,@rhogenelabata2774,2024-11-07 16:54:00,0.0,413,Harris,kentuckian republican here for harriswalz,Good Sense | Harris-Walz 2024,48831.0,en,positive


In [ ]:
from google.colab import files

# Download the CSV file
files.download('classified_comments.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
negative_comments = calssified_comments[calssified_comments['predicted_label'] == 'negative']


In [ ]:
# Save the subset to an Excel file
negative_comments.to_excel('negative_comments.xlsx', index=False)

# Download the file
files.download('negative_comments.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

continuing with annotating negative samples

In [ ]:
sampled_negative_comments = negative_comments.sample(n=1200, random_state=42)

sampled_negative_comments.to_excel('sampled_negative_comments.xlsx', index=False)
files.download('sampled_negative_comments.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>